In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Detect and initialize the TPU

In [ ]:

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()  # Default strategy that works on CPU and single GPU

print('Number of replicas:', strategy.num_replicas_in_sync)

# Access the GCS path for the Kaggle dataset

In [ ]:

GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
data_dir = GCS_DS_PATH + "/tfrecords-jpeg-192x192"

# Set image height, width, and batch size

In [ ]:

img_height, img_width = 192, 192
batch_size = 16 * strategy.num_replicas_in_sync

# Set up data input functions

In [ ]:

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [img_height, img_width, 3])
    return image

In [ ]:
def read_tfrecord(example, labeled=True):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.one_hot(example['class'], 104)
        return image, label
    return image, example['id']

def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = dataset.with_options(ignore_order)
    return dataset.map(lambda x: read_tfrecord(x, labeled), num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Load datasets

In [ ]:

train_files = tf.io.gfile.glob(data_dir + '/train/*.tfrec')
val_files = tf.io.gfile.glob(data_dir + '/val/*.tfrec')
test_files = tf.io.gfile.glob(data_dir + '/test/*.tfrec')

# Assuming the structure and size of the dataset are known:

In [ ]:

num_train_examples = 12753  # example value
num_val_examples = 3712     # example value
num_test_files = len(test_files)
num_test_examples = num_test_files * 2048  # Assuming batch size for test dataset is 2048
TRAIN_STEPS = num_train_examples // batch_size
VAL_STEPS = num_val_examples // batch_size

train_dataset = load_dataset(train_files).repeat().shuffle(2048).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = load_dataset(val_files).batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = load_dataset(test_files, labeled=False, ordered=True).batch(batch_size)

# Define the model architecture

In [ ]:

with strategy.scope():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(104, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model

In [ ]:

history = model.fit(train_dataset, steps_per_epoch=TRAIN_STEPS, validation_data=val_dataset, validation_steps=VAL_STEPS, epochs=10)

# Prediction and submission preparation

In [ ]:

def predict_and_prepare_submission(model, test_dataset):
    print('Predicting on test data...')
    predictions = model.predict(test_dataset)
    predicted_classes = np.argmax(predictions, axis=1)

    print('Generating submission file...')
    test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
    test_ids = next(iter(test_ids_ds.batch(num_test_examples))).numpy().astype('U')  # Adjust dtype for submission

    submission_df = pd.DataFrame({'id': test_ids, 'label': predicted_classes})
    submission_df.to_csv('submission.csv', index=False)

    print('Submission file created successfully!')

    return predicted_classes  # Return predicted classes

# Call the function to predict and prepare submission
predicted_classes = predict_and_prepare_submission(model, test_dataset)


# Predict and prepare submission

In [ ]:

predict_and_prepare_submission(model, test_dataset)

# Visualize training history

In [ ]:

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()